In [1]:
# stuff for generating phrase expansions and feature files
import collections, math, nltk, re, subprocess
import os
import yake
from plotlib.loaders import *
from phdconf import config
import itertools
import numpy as np
import annoy
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from typing import List, Dict
import math 
import requests

In [2]:
topics = load_queries('/home/danlocke/go/src/github.com/dan-locke/phd-data/case-topics.json')

In [3]:
# code for extracting phrases using yake ... 

# kw_extractor = yake.KeywordExtractor(lan='en', n=3, dedupLim=0.9, dedupFunc='seqm', windowsSize=1, top=40, features=None)

# exclude = {
#     "<i>", 
#     "</i>",
#     "<br>",
#     "<b>",
#     "</b>",
#     ',',
#     ';',
#     ':',
#     '\'',
#     '"',
# }

# path = '/home/danlocke/JSON/'

# PARSE_PATH = "/home/danlocke/go/src/parse/./parse"

# phrases = {}

# for root, d, files in os.walk(path):
# # files = ['/home/danlocke/JSON/FCA/2013/2013FCA0721.json']
#     for f in files:
#         if f.endswith('.json'):
#             print(f, end='\r')
#             res = subprocess.run([PARSE_PATH, os.path.join(root, f)], stdout = subprocess.PIPE).stdout
#             data = json.loads(res)
#             tokens = [x for x in list(itertools.chain(*data['paras'])) if x not in exclude]
#             if len(tokens) == 0:
#                 continue
#             keywords = kw_extractor.extract_keywords(' '.join(tokens))
#             for kw in keywords:
#                 if ' ' not in kw[1]:
#                     continue 
#     #                 print(kw)
#                 v = phrases.get(kw[1], 0)
#                 phrases[kw[1]] = v + 1

# phrases = {k: v for k, v in sorted(phrases.items(), key=lambda item: item[1], reverse=True)}
# with open('phrases.txt', 'w') as f:
#     for item in phrases.items():
#         f.write('{0} {1}\n'.format(item[0], item[1]))

In [4]:
exclude = {
    "__DAY__",
    "__DATE__",
    "__TIME__",
    "__MONTH__",

    "__CITATION__",
    "__CASENAME__",

    "__CLAUSE__",
    "__SECTIONREF__",
    "__LEGISLATION__",

    "__PINPOINT__",
    "__REFERENCE__",
    "__PARA__",
    "__ENUM__",

    "__JUDGE__",
    "__COURT__",

    "__PERSON__",
    "__PARTY__",

    "__COMPANY__",
    "__ACN__",
    "__ABN__",

    "__ENTITY__",

    "__MONEY__",
    "__PERCENTAGE__",

    "__GROUND__",
    "__EXHIBIT__",
    "__PLEADING__",
    "__TRANSCRIPT__",
    "__APPEALBUNDLE__",
    "__DECNO__",
    "__FILENO__",
    "__REPORT__",
    "__ORDER__",
    "__SECONDARY__",
    "<i>",
    "</i>",
    "<br>",
    "<b>",
    "</b>",
    "<u>",
    "</u>",
    "<ref>",
    "</ref>",
    "[",
    "]",
    ",",
    "\\",
    "/",
    "\"",
    ".",
    "_",
    "||",
    "%",
}

stop = {
    'the',
    'of',
    'to',
    'in', 
    'for',
    'that',
    'and',
    'on',
    'is',
    'be',
    'by',
    'a',
    'an',
    'was',
    'it',
    'as',
    'this',
    'which',
    'with',
    'have',
    'at',
    'been',
    'there',
    'no',
    'or',
    'from',
    'has',
    'any',
    'i',
    'would',
    'were',
    'had',
    'are',
    'if',
    'also',
    'before',
    'but',
    'his',
    'other',
    'those',
    'so',
    'he',
    'did',
    'its',
    'she',
    'hers',
    'their',
    'theirs'
}

In [5]:
def read_phrase_file(path: str): 
    phrases = {}
    with open(path) as f:
        for line in f:
            if '.' in line:
                continue
            parts = line.split()
            skip = False 
            
            for t in parts[:-1]:
                if t in exclude or not t.isalpha():
                    skip = True
                    break 

            if skip:
                continue


            tokens = [x for x in parts[:-1] if x not in stop]
            p = ' '.join(tokens).replace('-', ' ')
            v = phrases.get(p, 0)
            phrases[p] = v + int(parts[-1])
            
    return phrases

In [6]:
# read phrase file in embedding training format
def load_phrase_embedding_file(path: str) -> Dict[str, int]:
    phrases = {}
    with open(path) as f:
        for line in f: 
            parts = line.split()
            for part in parts:
                v = phrases.get(part, 0)
                phrases[part] = v+1
    return phrases

In [7]:
phrases = load_phrase_embedding_file('/home/danlocke/go/src/doc_phrases/coll-phrases.txt')
# phrases = read_phrase_file('/home/danlocke/src/phd-notebooks/phrases.txt')
# phrases = read_phrase_file('/home/danlocke/go/src/coll_phrases/coll-phrases.txt')
# phrases = [x for x, y in phrases.items() if y > 5] 

In [8]:
# [x for x in phrases if x.startswith('act_')]
len(phrases)

2745417

In [9]:
find = {
    'first', 
    'second', 
    'third',
    'fourth',
    'fifth',
    'sixth',
    'seventh',
    'eighth',
    'ninth',
    'tenth',
    'eleventh',
    'twelfth',
    'thirteenth',
    'fourteenth',
    'fifteenth',
    'sixteenth',
    'seventeenth',
    'eighteenth',
    'nineteenth',
    'twenty',
    'thirty',
    'fourty',
}

num_exclusion = {
    'amendment',
    'degree',
    'refusal'
}

find2 = {
    'north',
    'south',
    'east',
    'west'
}

find3 = {
    'set',
    'hold',
    'exemplary',
    'intending',
    'include',
    'including',
    'relies',
    'relied',
    'pursuant',
    'thereto',
    'behalf',
    'giving',
    'give',
    'apparently',
    'subsequently',
    'jurisdictional',
    'included',
    'includes',
    'appellant',
    'respondent',
    'resulting',
    'such',
    's',
    'x',
    't',
    'th',
    'thing',
    're',
    'subject'
}

find4 = {
    'nonetheless',
    'follows',
    'referred',
    'thereto',
    'behalf',
    'hastily',
    'instance',
    'instances',
    'such',
    's',
    'applicant',
    'applicants',
    'respondent',
    'respondents',
    'th',
    'x',
    'such',
    'll',
    'only',
    'way',
    're',
    'much',
    'st',
}

#     consider 'made' and 'make' as exclusions

def filter_phrases(phrases, splitter=' '):
    d = isinstance(phrases, dict)
    if d:
        ret = {}
    else:
        ret = []
    for phrase in phrases:
        toks = phrase.split(splitter)
        if len(toks) < 2:
            continue 
        
        cont = False 
        cnt = 0

        if toks[-1] in find3 or len(toks[-1]) == 1:
            continue
        
        if len(toks) == 2 and toks[0].startswith('claim'):
            continue
            
        if len(toks) == 3 and toks[0] == 'intention' and toks[1] != 'to':
            continue
            
        if toks[0] == 'intention' and len(toks) == 2:
            continue
            
        if toks[0] in find4 or len(toks[0]) == 1:
            continue
            
        unique = set()
            
        num = False 
        num_ex = False
        for t, tok in enumerate(toks):
            unique.add(tok)
            
            if tok in find: 
                num = True 
            if tok in num_exclusion:
                num_ex = True
            if tok in find2:
                cnt += 1 
            if tok == 'nunc' and t != 0:
                cont = True 
                break
                
        if num and not num_ex: 
            continue
        
        if len(toks) > len(unique): 
            continue
    
        if cont or cnt > 1:
            continue 
            
        if d: 
            ret[phrase] = phrases[phrase]
        else:
            ret.append(phrase)
        
    return ret 

In [10]:
# filtered_phrases = filter_phrases(phrases, '_')

In [11]:
# filtered_phrases += [
#     'set off', 
#     'res judicatum', 
#     'res judicatums', 
#     'suspension clauses', 
#     'suspension clause', 
#     'right first refusal',
#     'equitable jurisdiction',
#     'expectation losses',
#     'non party costs',
#     'strike out',
#     'striking out',
#     'party party costs', # this gets removed by a filtering rule 
# ]

In [12]:
# [x for x in filtered_phrases if x.startswith('applicant')]
# len(filtered_phrases)
# [x for x in filtered_phrases if 'first' in x ]

In [13]:
def damerau_levenshtein_distance(s1, s2):
    for i in range(-1, len(s1) + 1):
        d[i, -1] = i + 1
    for j in range(-1, len(s2) + 1):
        d[-1, j] = j + 1

    for i, cs1 in enumerate(s1):
        for j, cs2 in enumerate(s2):
            cost = int(not self.test_func(cs1, cs2))
            # ^ 0 if equal, 1 otherwise

            d[i, j] = min(
                d[i - 1, j] + 1,            # deletion
                d[i, j - 1] + 1,            # insertion
                d[i - 1, j - 1] + cost,     # substitution
            )

            # transposition
            if not i or not j:
                continue
            if not self.test_func(cs1, s2[j - 1]):
                continue
            if not self.test_func(s1[i - 1], cs2):
                continue
            d[i, j] = min(
                d[i, j],
                d[i - 2, j - 2] + cost,
            )

    return d[len(s1) - 1, len(s2) - 1]

In [14]:
def load_coll_stats(path: str, total=None) -> Dict[str, float]:
    stats = {}
    
    idf = total is None
    if idf:
        total = 0.0
    with open(path) as f:
        for line in f:
            parts = line.split()
            if idf: 
                v = float(parts[-1])
                total += v
                stats[' '.join(parts[:-1])] = v
            else:
                stats[' '.join(parts[:-1])] = math.log(total / float(parts[-1])+1.0) + 1.0
    
    if idf: 
        for k, v in stats.items():
            stats[k] = math.log(total / v+1.0) + 1.0

    return stats

In [15]:
coll_stats = load_coll_stats('/home/danlocke/phd-generated/filtered-stop-top-tokens.txt')

In [16]:
def load_vectors(path: str):
    embeddings = {}
    with open(path, 'r', encoding='utf-8') as f:
        f.readline()
        for line in f:
            values = line.rstrip().rsplit(' ')
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings[word] = coefs

    return embeddings

embeddings = load_vectors('/home/danlocke/fastText/filtered-100d.vec')

class EmbMethod:
    SUM = 0
    MEAN = 1 
    CF_SUM = 2 
    CF_MEAN = 3

def embed(tokens: List[str], embeddings, dim: int=100, method: EmbMethod = EmbMethod.SUM, coll_stats = None) -> np.array:
    e = []
    if method > EmbMethod.MEAN: 
        for x in tokens: 
            if x in embeddings:
                mul = coll_stats[x] if x in coll_stats else 1.0
                e.append(mul * embeddings[x])
    else: 
        e = [embeddings[x] for x in tokens if x in embeddings]
    if len(e) == 0:
        return np.zeros((dim,), dtype='float32')
    
    e = np.sum(e, axis=0)
    if method == EmbMethod.MEAN or method == EmbMethod.CF_MEAN:
        e /= len(tokens)
            
    return e

In [17]:
embeddings = {k: v for k, v in embeddings.items() if k.islower() and k.isalpha()}
term_embeddings = [x for x in embeddings.values()]
term_lookup = [x for x in embeddings.keys()]

In [18]:
generic_embeddings = load_vectors('/home/danlocke/wiki-news-300d-1M.vec')

In [18]:
phrase_embeddings = load_vectors('/home/danlocke/fastText/para-phrase-100d.vec')

In [19]:
len(phrase_embeddings)

431399

In [20]:
# [k for k in phrase_embeddings.items() if k[1].shape[0] == 13]

phrase_embeddings = filter_phrases(phrase_embeddings, '_')
phrase_embeddings_vecs = [x for x in phrase_embeddings.values()]
phrase_lookup = [x for x in phrase_embeddings.keys()]
# for i, x in enumerate(phrase_embeddings):
# len(phrase_embeddings)
# list(phrase_embeddings.items())[0][1].shape

In [21]:
# [x for x in phrase_lookup if x.startswith('start_')]
# start much 
# start/end subject 

In [22]:
ind = annoy.AnnoyIndex(100, 'angular')
for i, x in enumerate(phrase_embeddings_vecs):
    ind.add_item(i, x) 
ind.build(1024)
# ind.addDataPointBatch(phrase_embeddings)
# ind.createIndex({"post": 0, 'indexThreadQty':23, 'M': 64, 'efConstruction': 5000})

True

In [23]:
def get_phrases(text: str, phrase_lookup: Dict[str, np.array]): 
    found = {}
    tokens = [x for x in text.lower().replace(',', '').replace('?', '').replace('\'', '').replace('/', '').replace('’', '').split() if x not in stop]
    for grams in [n_gram(tokens, 2), n_gram(tokens, 3)]:
        for gram in grams:
            if gram in phrase_lookup:
                found[gram] = 1
                
    return found

In [35]:
def n_gram(tokens: List[str], n: int): 
    ret = []
    for i in range(0, len(tokens)-(n-1)):
           ret.append('_'.join(tokens[i:i+n]))
           
    return ret 

meth = EmbMethod.SUM

cnt = 0

for topic in topics.values(): 
    print('-'*50)
    print(topic['topic'])
    found = get_phrases(topic['topic'], phrase_embeddings)
    tokens = [x for x in topic['topic'].lower().replace(',', '').replace('?', '').replace('\'', '').replace('/', '').replace('’', '').split() if x not in stop]
    
    qry_vec = embed(tokens, embeddings, method=meth, coll_stats=coll_stats).reshape(1, -1)
#     qry_generic_vec = embed(tokens, generic_embeddings, method=meth, coll_stats=coll_stats).reshape(1, -1)
    print(found)
    for f in found:
        f_t = f.split('_')
#         e = embed(f_t, embeddings, method=meth, coll_stats=coll_stats)
        orig_constructed_vec = embed(f_t, embeddings, method=meth, coll_stats=coll_stats).reshape(1, -1)
        e = phrase_embeddings[f]
#         f_t = [x for x in f_t if x not in stop]
        f_t_map = {x:1 for x in f_t}
        res = ind.get_nns_by_vector(e, 25, include_distances=True)
#         print('*'*10)
#         print(f)
        for i, x in enumerate(res[0]):
#             if i == 0:
#                 continue
#             if f in filtered_phrases[x]:
#                 continue
#             p_t = [t for t in filtered_phrases[x].split() if t not in stop]
            p_t = phrase_lookup[x].split('_')
            if p_t == f_t:
                continue
                
            remaining = [y for y in p_t if y not in f_t_map]
            if '_'.join(remaining) in f:
                continue 
            p_t_vec = embed(p_t, embeddings, method=meth, coll_stats=coll_stats).reshape(1, -1)
#             phrase_sim = cosine_similarity(qry_vec, phrase_embeddings[x:x+1])[0][0] 
            phrase_vec_sim = cosine_similarity(e.reshape(1, -1), phrase_embeddings_vecs[x:x+1])[0][0]
            phrase_sim = cosine_similarity(orig_constructed_vec, p_t_vec)[0][0] 
            
#             qry_phrase_sim = cosine_similarity(qry_vec, phrase_embeddings[x:x+1])[0][0]
            qry_phrase_sim = cosine_similarity(qry_vec, p_t_vec)[0][0] 

            dist = nltk.edit_distance(' '.join(f_t), ' '.join(p_t))
            
#             generic_qry_sim = cosine_similarity(qry_generic_vec, embed(p_t, generic_embeddings, 300, meth, coll_stats=coll_stats).reshape(1, -1))[0][0]
            remaining_sim = cosine_similarity(orig_constructed_vec, embed(remaining, embeddings, method=meth, coll_stats=coll_stats).reshape(1, -1))[0][0]
#             generic_remaining_sim = cosine_similarity(embed(f_t, generic_embeddings, 300, method=meth, coll_stats=coll_stats).reshape(1, -1), embed(remaining, generic_embeddings, 300, meth, coll_stats=coll_stats).reshape(1, -1))[0][0]
            
            keep = True

            if phrase_sim < 0.8 and dist > 2:
                keep = False
#                 continue 
                
            if phrase_sim < 0.7:
                keep = False 
#                 continue
                
            if remaining_sim < 0.5:
                keep = False
#                 continue
                
            if len(remaining) == 0:
                keep = True
#             if diff_token_sim < 0.6 or res[1][i] > 0.6:
#                 continue
                
            print('{7:30} {0:30} {1:25} {2:.4f} {3:.4f} {4:4} {5:.4f} {6:.4f} {8}'.format(phrase_lookup[x],
                  ' '.join(remaining), 
#                   res[1][i], 
                  phrase_vec_sim, 
                  phrase_sim,
                  qry_phrase_sim, 
                  dist,
                  remaining_sim, 
                  f,
                  keep, 
                  ))

#             print('{8} {0} {9:d}'.format(phrase_lookup[x],
# #                     filtered_phrases[x], 
#                   ' '.join(remaining), 
#                   res[1][i], 
#                   qry_phrase_sim, 
#                   dist,
#                   generic_qry_sim,
#                   remaining_sim, 
#                   generic_remaining_sim,
#                   f.replace(' ', '_'),
#                   keep          
#                   ))
# #         print('====')
#         e = e.reshape(1, -1)
#         term_sims = np.asarray([cosine_similarity(e, term_embeddings[x:x+1])[0][0] for x in range(len(term_embeddings))])
#         sims = term_sims.argsort()[:-10:-1]
#         for x in sims:
#             if term_lookup[x].lower() in f_t_map:
#                 continue
#             sim_val = cosine_similarity(qry_vec, term_embeddings[x:x+1])[0][0]
#             if sim_val < 0.7:
#                 continue 
#             print(term_lookup[x], term_sims[x], sim_val)
        
    if cnt > 8:
        break
    cnt += 1
        
            
#     keywords = kw_extractor.extract_keywords(topic['topic'])
#     for kw in keywords:
#         if ' ' not in kw[1] or kw[1] in found or ' and ' in kw[1]:
#             continue 
#         print(kw)


--------------------------------------------------
What is the effect of reinstating a company that was in liquidation as regards money that may be recovered?
{'reinstating_company': 1, 'company_liquidation': 1}
reinstating_company            suing_company                  suing                     0.8521 0.7635 0.7768868207931519 7.0000 0.4905 False
reinstating_company            bring_company                  bring                     0.8512 0.7649 0.7650007009506226 8.0000 0.3880 False
reinstating_company            reinstate_company              reinstate                 0.8433 0.9402 0.758665919303894 3.0000 0.7465 True
reinstating_company            keeping_company                keeping                   0.8413 0.7674 0.7248092889785767 7.0000 0.3590 False
reinstating_company            giving_company                 giving                    0.8365 0.7344 0.7674388885498047 7.0000 0.2657 False
reinstating_company            abetting_company               abetting               

In [121]:
def load_index(index_name: str, index_path: str):
    base = 'http://127.0.0.1:8080/_load'
    req = requests.post(base, json={'index': index_name, 'path': index_path})
    data = json.loads(req.text)
    print(data)
    req = requests.post('http://127.0.0.1:8080/settings', 
                        json={'field': 'text', 'index': 'filtered-stop', 'scoring_method': 'unigram', 'term_scoring_method': 'dir', 
                              'term_scoring_params': {'mu': 2400.0}, 'tokenizer': 'html_tokenizer', 'lower': True, 'ignore_punc': True})
    data = json.loads(req.text)
    print(data)
    
load_index('filtered-stop', '/Users/danlocke/Desktop/')

{'message': "Index: 'filtered-stop' loaded."}
{'message': ''}


In [304]:
def query(query: str, index:str='filtered-stop'):
    req = requests.post('http://127.0.0.1:8080/search', json={'query': query, 'field': 'text', 'params': {}, 'index': index})
    data = json.loads(req.text)
    return data['docs']

def score_phrase(phrase: str, docs: List[int], index:str='filtered-stop') -> List[float]:
    req = requests.post('http://127.0.0.1:8080/score_phrase', json={'docs': docs, 'query': phrase, 'field': 'text', 'params': {}, 'index': index})
    data = json.loads(req.text)
    return data['scores']

def combine_scores(query_results, scores: List[float], interp=0.8):
    for i, res in enumerate(query_results):
        query_results[i]['score'] = interp * query_results[i]['score'] + (1.0 - interp) * scores[i]
    return query_results

def get_ids(docs: List[str], index:str='filtered-stop'):
    req = requests.post('http://127.0.0.1:8080/ids', json={'docs': docs, 'index': index})
    data = json.loads(req.text)
    return data['Ids']

def generate_res_file(results, topic_id: str):
    with open('tmp.run', 'w') as f:
        for i, res in enumerate(results): 
            f.write('{0} Q0 {1} {2} {3} t\n'.format(topic_id, res['id'], i, res['score']))
            
def delete_res_file():
    os.remove('tmp.run')
    
def compare_to_baseline(results, topic_id: int, baseline):
    generate_res_file(results, topic_id)
    res = load_dfs(config.AUS_QREL_PATH, '1', '', ['tmp.run'], per_query=True)[0]
    return res.loc[topic_id] - baseline
    
def load_qrels(path: str):
    qrels = {}
    with open(path) as f:
        for line in f:
            parts = line.strip().split()
            if parts[0] not in qrels:
                qrels[parts[0]] = set()
            qrels[parts[0]].add(parts[2])
            
    return qrels
# delete_res_file()

In [205]:
import os
# jm_path = os.path.join(config.BASE_DIR, 'jelinek_mercer')
BASE_DIR = os.path.join(os.environ["HOME"], 'phd-generated')
index_names = ['filtered-stop']
qrel_paths = [config.AUS_QREL_PATH]
rel_levels =['1']
baseline_df = load_1d_dfs(index_names, qrel_paths, os.path.join(BASE_DIR, 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, 2400, 2401, 1, per_query=True)[0][0]

In [210]:
qrels = load_qrels(config.AUS_QREL_PATH)

In [324]:
len(topics)

95

In [341]:
def evaluate_possible_phrases(f, topics, topic_id: int, phrases: List[str], qrels):
    qry = topics[topic_id]['topic']
    query_res = query(qry)
#     print(qry)
    docs = [x['id'] for x in query_res]
#     print([x['id'] for x in query_res[:10] if x['id'] not in qrels[str(topic_id)]])
    ids = get_ids(docs)
    ids, query_res, docs = zip(*sorted(zip(ids, query_res, docs)))
    num_tok = len([x for x in qry.lower().replace(',', '').replace('?', '').replace('\'', '').replace('/', '').replace('’', '').split() if x not in stop])
    interp = 1.0 / float(num_tok)
    unjudged = set()
#     print(interp, len(phrases), num_tok)
    for phrase in phrases: 
        phrase_scores = score_phrase(phrase.replace('_', ' '), docs)
#         print(phrase_scores)
#         print([x['score'] for x in query_res])
        res = combine_scores(query_res, phrase_scores, interp)
        res = sorted(res, key=lambda item: item['score'], reverse=True) 
#         print(res)
        res = compare_to_baseline(res, topic_id, baseline_df.loc[topic_id])

#         for x in query_res[:10]: 
#             if x['id'] not in qrels[str(topic_id)]:
#                 unjudged.add(x['id'])
#         print('{0} {1} {2:.4f} {3:.4f} {4:.4f} {5:.4f} {6:.4f} {7:.4f} {8:.4f}'.format(topic_id, phrase, res['recip_rank'], res['recall_20'], res['recall_100'], res['ndcg'], res['rbp@0.10'], res['rbp@0.50'], res['rbp@0.80']))
        f.write('{0} {1} {2:.4f} {3:.4f} {4:.4f} {5:.4f} {6:.4f} {7:.4f} {8:.4f}\n'.format(topic_id, phrase, res['recip_rank'], res['recall_20'], res['recall_100'], res['ndcg'], res['rbp@0.10'], res['rbp@0.50'], res['rbp@0.80']))
      
#     for doc in unjudged: 
#         print('{0} 0 {1} 0'.format(topic_id, doc))
    
with open('expansion-changes.txt', 'w') as f:
    for topic in topics.keys():
        qry_phrases = get_phrases(topics[topic]['topic'], phrase_embeddings)
        qry_phrases = set(qry_phrases.keys())

        phrases = set()
        for phrase in qry_phrases:
            res = ind.get_nns_by_vector(phrase_embeddings[phrase], 20)
            for x in res:
                phrases.add(phrase_lookup[x])

        phrases = qry_phrases.union(phrases)
        evaluate_possible_phrases(f, topics, topic, phrases, qrels)

In [ ]:
# filter training file 
def filter_phrase_embedding_file(path: str, out_path: str):
    out_file = open(out_path)
    with open(path) as f:
        for line in f: 
            phrases = filter_phrases(line.split(), '_')
            if len(phrases) != 0:
                out_file.write('{0}\n'.format(' '.join(phrases)))
    
    out_file.close()

filter_phrase_embedding_file('~/go/src/doc_phrases/doc-phrases.txt', 'filtered-phrases.txt')